# 循环神经网络（RNN）文本生成

本教程演示如何使用基于字符的 RNN 生成文本。我们将使用 Andrej Karpathy 在《循环神经网络不合理的有效性》一文中提供的莎士比亚作品数据集。给定此数据中的一个字符序列 （“Shakespear”），训练一个模型以预测该序列的下一个字符（“e”）。通过重复调用该模型，可以生成更长的文本序列。

本教程包含使用 tf.keras 和 eager execution 实现的可运行代码。以下是当本教程中的模型训练 30 个周期 （epoch），并以字符串 “Q” 开头时的示例输出：

虽然有些句子符合语法规则，但是大多数句子没有意义。这个模型尚未学习到单词的含义，但请考虑以下几点：
+ 此模型是基于字符的。训练开始时，模型不知道如何拼写一个英文单词，甚至不知道单词是文本的一个单位。

+ 输出文本的结构类似于剧本 -- 文本块通常以讲话者的名字开始；而且与数据集类似，讲话者的名字采用全大写字母。

+ 如下文所示，此模型由小批次 （batch） 文本训练而成（每批 100 个字符）。即便如此，此模型仍然能生成更长的文本序列，并且结构连贯。

## 导入库

In [8]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import numpy as np
import os
import time

## 下载莎士比亚数据集

In [9]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 1us/step


## 读取数据

In [10]:
# 读取并为 .py2 compat解码
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(text[:20])
# 文本长度是指文本中的字符个数
print("Length of text:{} characters".format(len(text)))

First Citizen:
Befor
Length of text:1115394 characters


In [11]:
# 看一下文本中的前250个字符
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [35]:
# 文本中非重复字符
vocab = sorted(set(text))
print(type(text))
print(type(vocab))
print("{} unique characters".format(len(vocab)))

<class 'str'>
<class 'list'>
65 unique characters


## 处理文本

向量化文本

在训练之前，我们需要将字符串映射到数字表示值。创建两个查找表格：一个将字符映射到数字，另一个将数字映射到字符。

In [14]:
# 创从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text]) # 转化为数字

现在，每个字符都有一个整数表示值。请注意，我们将字符映射至索引 0 至 len(unique)

In [17]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("  {:4s}: {:3d},".format(repr(char), char2idx[char]))

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,


In [19]:
# 显示文本首 13 个字符的整数映射

print("{} ---- character mapped to int --- > {}".format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- character mapped to int --- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## 预测任务

给定一个字符或者一个字符序列，下一个最可能出现的字符是什么？这就是我们训练模型要执行的任务。输入进模型的是一个字符序列，我们训练这个模型来预测输出 -- 每个时间步（time step）预测下一个字符是什么。

由于 RNN 是根据前面看到的元素维持内部状态，那么，给定此时计算出的所有字符，下一个字符是什么？

## 创建训练样本和目标

接下来，将文本划分为样本序列。每个输入序列包含文本中的 seq_length 个字符。

对于每个输入序列，其对应的目标包含相同长度的文本，但是向右顺移一个字符。

将文本拆分为长度为 seq_length+1 的文本块。例如，假设 seq_length 为 4 而且文本为 “Hello”， 那么输入序列将为 “Hell”，目标序列将为 “ello”。

首先使用 tf.data.Dataset.from_tensor_slices 函数把文本向量转换为字符索引流。

In [20]:
# 设定每个句子长度的最大值
seq_length = 100
example_per_epoch = len(text) // seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


batch 方法使我们能轻松把单个字符转换为所需长度的序列。

In [22]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True) # frop_remainder 表示最后剩余的data数量不满足一个batch则丢弃

for item in sequences.take(5):
    print(item)
    print(repr("".join(idx2char[item.numpy()])))

tf.Tensor(
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1], shape=(101,), dtype=int32)
'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
tf.Tensor(
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1 49], shape=(101,), dtype=int32)
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
tf.Tensor(
[52 53 61  1 15 39 47 59 57  1 25 39 56 41 47 59 57  1 47 57  1 41 46 47

对于每个序列，使用 map 方法先复制再顺移，以创建输入文本和目标文本。map 方法可以将一个简单的函数应用到每一个批次 （batch）。

In [25]:
def split_input_target(chunk):
    """获得输入文本和目标文本"""
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [28]:
# 打印第一批样本的输入和输出

for input_example, target_example in dataset.take(1):
    print("Input data:", repr("".join(idx2char[input_example.numpy()])))
    print("Target data:", repr("".join(idx2char[target_example.numpy()])))

Input data: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


这些向量的每个索引均作为一个时间步来处理。作为时间步 0 的输入，模型接收到 “F” 的索引，并尝试预测 “i” 的索引为下一个字符。在下一个时间步，模型执行相同的操作，但是 RNN 不仅考虑当前的输入字符，还会考虑上一步的信息。

In [32]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


## 创建训练批次

前面我们使用 tf.data 将文本拆分为可管理的序列。但是在把这些数据输送至模型之前，我们需要将数据重新排列 （shuffle） 并打包为批次。

In [33]:
# 批次大小
BATCH_SIZE = 64

#  设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列）
# 所以它并不会试图在内存中重新排列整个序列，相反，
# 它维持一个缓冲区，在缓冲区重新排列元素

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [34]:
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

## 创建模型

使用 tf.keras.Sequential 定义模型。在这个简单的例子中，我们使用了三个层来定义模型：

+ tf.keras.layers.Embedding：输入层。一个可训练的对照表，它会将每个字符的数字映射到一个 embedding_dim 维度的向量。
+ tf.keras.layers.GRU：一种 RNN 类型，其大小由 units=rnn_units 指定（这里你也可以使用一个 LSTM 层）。
+ tf.keras.layers.Dense：输出层，带有 vocab_size 个输出。

In [36]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [63]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer="glorot_uniform"),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [64]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size = BATCH_SIZE
)

对于每个字符，模型会查找嵌入，把嵌入当作输入运行 GRU 一个时间步，并用密集层生成逻辑回归 （logits），预测下一个字符的对数可能性

![model](https://github.com/littlebeanbean7/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

## 检查模型

In [65]:
for input_example_batch, target_example_batch in dataset.take(1):
    print(input_example_batch.shape)
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100)
(64, 100, 65) # (batch_size, sequence_length, vocab_size)


在上面的例子中，输入的序列长度为 100， 但是这个模型可以在任何长度的输入上运行：

In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


为了获得模型的实际预测，我们需要从输出分布中抽样，以获得实际的字符索引。这个分布是根据对字符集的逻辑回归定义的。

请注意：从这个分布中 抽样 很重要，因为取分布的 最大值自变量点集（argmax） 很容易使模型卡在循环中。

试试这个批次中的第一个样本：

In [67]:
#  tf.random.categorical 说明 https://blog.csdn.net/a845717607/article/details/99701349
# tf.squeeze 说明 https://www.jianshu.com/p/a21c0bc10a38

In [68]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
print(sampled_indices.shape)
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()
print(sampled_indices.shape)

(100, 1)
(100,)


In [69]:
sampled_indices

array([31, 29, 22, 48, 18, 34, 12, 32, 52, 38,  1,  2, 29, 10, 49, 14, 33,
        4,  9, 35, 34, 36, 64, 45, 40,  5, 26, 45, 52,  0, 35, 63,  7, 22,
       34, 48, 32, 36, 17, 19, 54, 62, 57, 34, 17, 15, 25, 41, 43, 37,  7,
       41, 28, 15, 47, 31, 44, 14, 34, 13,  0, 15, 22, 11, 27, 23, 30, 28,
       62,  5, 24, 36, 33, 33, 23,  2, 44, 10, 25,  5, 39, 43, 18, 59, 30,
       49, 10, 58,  1, 50, 31, 53, 29, 20, 62,  8, 36, 58, 12, 29],
      dtype=int64)

In [70]:
# 解码它们，以查看此未经训练的模型预测的文本：
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Prediction: \n", repr("".join(idx2char[sampled_indices])))

Input: 
 'on,\nWho is already sick and pale with grief,\nThat thou her maid art far more fair than she:\nBe not h'

Next Char Prediction: 
 "SQJjFV?TnZ !Q:kBU&3WVXzgb'Ngn\nWy-JVjTXEGpxsVECMceY-cPCiSfBVA\nCJ;OKRPx'LXUUK!f:M'aeFuRk:t lSoQHx.Xt?Q"


## 训练模型

添加优化器和损失函数

标准的 tf.keras.losses.sparse_categorical_crossentropy 损失函数在这里适用，因为它被应用于预测的最后一个维度。

因为我们的模型返回逻辑回归，所以我们需要设定命令行参数 from_logits。

In [71]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1738343


使用 tf.keras.Model.compile 方法配置训练步骤。我们将使用 tf.keras.optimizers.Adam 并采用默认参数，以及损失函数。

In [72]:
model.compile(optimizer="adam", loss=loss)

### 配置检查点

使用 tf.keras.callbacks.ModelCheckpoint 来确保训练过程中保存检查点。

In [76]:
# 检查点保存至目录
checkpoint_dir = "./training_checkpoints"

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "cpk_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

### 执行训练

为保持训练时间合理，使用 10 个周期来训练模型

In [77]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 27s 158ms/step - loss: 1.9628
Epoch 2/10
172/172 [==============================] - 27s 157ms/step - loss: 1.6971
Epoch 3/10
172/172 [==============================] - 27s 157ms/step - loss: 1.5458
Epoch 4/10
172/172 [==============================] - 27s 157ms/step - loss: 1.4569
Epoch 5/10
172/172 [==============================] - 27s 158ms/step - loss: 1.3974
Epoch 6/10
172/172 [==============================] - 27s 157ms/step - loss: 1.3515
Epoch 7/10
172/172 [==============================] - 27s 158ms/step - loss: 1.3117
Epoch 8/10
172/172 [==============================] - 27s 157ms/step - loss: 1.2776
Epoch 9/10
172/172 [==============================] - 27s 157ms/step - loss: 1.2453
Epoch 10/10
172/172 [==============================] - 27s 159ms/step - loss: 1.2116


## 生成文本

恢复最新的检查点

为保持此次预测步骤简单，将批大小设定为 1。

由于 RNN 状态从时间步传递到时间步的方式，模型建立好之后只接受固定的批大小。

若要使用不同的 batch_size 来运行模型，我们需要重建模型并从检查点中恢复权重。

In [78]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\cpk_10'

In [79]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [80]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## 预测循环

下面的代码块生成文本：

+ 首先设置起始字符串，初始化 RNN 状态并设置要生成的字符个数。

+ 用起始字符串和 RNN 状态，获取下一个字符的预测分布。

+ 然后，用分类分布计算预测字符的索引。把这个预测字符当作模型的下一个输入。

+ 模型返回的 RNN 状态被输送回模型。现在，模型有更多上下文可以学习，而非只有一个字符。在预测出下一个字符后，更改过的 RNN 状态被再次输送回模型。模型就是这样，通过不断从前面预测的字符获得更多上下文，进行学习。

查看生成的文本，你会发现这个模型知道什么时候使用大写字母，什么时候分段，而且模仿出了莎士比亚式的词汇。由于训练的周期小，模型尚未学会生成连贯的句子。

In [82]:
def generate_text(model, start_string):
    # 评估步骤（用学习过的模型生成文本）

    # 要生成的字符个数
    num_generate = 1000
    
    # 将起始字符串转换为数字（向量化）
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    # 空字符串用于存储结果
    text_generated = []
    
    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0
    
    # 批次大小
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
        
        # 用分类分布预测模型返回的字符
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        
        # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
    
    return (start_string + "".join(text_generated))

In [83]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: I will be fit to have prevails, or now
That all this wounded health showers as if I loose a forfeit.
I thought of than fear is now,
That no more object makes not denied thy brother's lord?
What's there no grace crecyable and riverend
humafter than my boys i' the show impatience.

DUKE VINCENTIO:
O isparlour sorrow arriage?

ISABELLA:
I am for our creatures yield nt on at innexts,
If that shall gafied o' the field leas' pariledo man.

CLOFID such a horse!

JULIET:
Do so! no, 'tis not a heavy lord.

YORKE:
I am not tobut?

BAPster:
If you arm; but there's married,
I would they set upon my honourable, King Henry of Barnardine.

Servant:
Bud, go to:
I, that is not afterity of requits nothing,
That, like revenues mecauld they vault hid not to be dined:
For each was to try him, I hear the postern of all,
And in your eyes from viPbids in my heart itsed judge
Would I did true his silence allowing true death!

DUCHESS OF YORK:
O, call them past the time to have from me.

CATESBY:
Go to F

若想改进结果，最简单的方式是延长训练时间 （试试 EPOCHS=30）。

你还可以试验使用不同的起始字符串，或者尝试增加另一个 RNN 层以提高模型的准确率，亦或调整温度参数以生成更多或者更少的随机预测。

## 高级：自定义训练

上面的训练步骤简单，但是能控制的地方不多。

至此，你已经知道如何手动运行模型。现在，让我们打开训练循环，并自己实现它。这是一些任务的起点，例如实现 课程学习 以帮助稳定模型的开环输出。

你将使用 tf.GradientTape 跟踪梯度。

步骤如下：
+ 首先，初始化 RNN 状态，使用 tf.keras.Model.reset_states 方法。

+ 然后，迭代数据集（逐批次）并计算每次迭代对应的 预测。

+ 打开一个 tf.GradientTape 并计算该上下文时的预测和损失。

+ 使用 tf.GradientTape.grads 方法，计算当前模型变量情况下的损失梯度。

+ 最后，使用优化器的 tf.train.Optimizer.apply_gradients 方法向下迈出一步。

In [84]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [85]:
optimizer = tf.keras.optimizers.Adam()

In [86]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [90]:
# 训练步骤
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    # 在每个训练周期开始时，初始化隐藏状态
    # 隐藏状态最初为None
    hidden = model.reset_states()
    
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
        
        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))
            
    # 每 5 个训练周期，保存（检查点）1 次模型
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 1.3900631666183472
Epoch 1 Batch 100 Loss 1.362947702407837
Epoch 1 Loss 1.4101
Time taken for 1 epoch 23.434309482574463 sec

Epoch 2 Batch 0 Loss 1.3035521507263184
Epoch 2 Batch 100 Loss 1.3716347217559814
Epoch 2 Loss 1.3856
Time taken for 1 epoch 25.36791229248047 sec

Epoch 3 Batch 0 Loss 1.2720381021499634
Epoch 3 Batch 100 Loss 1.3098969459533691
Epoch 3 Loss 1.3156
Time taken for 1 epoch 165.90118169784546 sec

Epoch 4 Batch 0 Loss 1.2547330856323242
Epoch 4 Batch 100 Loss 1.2895861864089966
Epoch 4 Loss 1.2978
Time taken for 1 epoch 191.35083055496216 sec

Epoch 5 Batch 0 Loss 1.1835075616836548
Epoch 5 Batch 100 Loss 1.2099910974502563
Epoch 5 Loss 1.2707
Time taken for 1 epoch 307.5058603286743 sec

Epoch 6 Batch 0 Loss 1.133288860321045
Epoch 6 Batch 100 Loss 1.2299422025680542
Epoch 6 Loss 1.2113
Time taken for 1 epoch 352.7068145275116 sec

Epoch 7 Batch 0 Loss 1.1225453615188599
Epoch 7 Batch 100 Loss 1.2066569328308105
Epoch 7 Loss 1.2522
Time take